In [1]:
# load data from tsp file
import numpy as np

#tsp_data = np.loadtxt('C:\IMT_2023-2024\ISEN\Enseignement\TSPDataset\\gr17.2085.tsp')#fri26_d.937 gr17.2085
tsp_data = np.loadtxt('../TSPDataset/gr17.2085.tsp')  
print(tsp_data.shape)

(17, 17)


In [ ]:
import math
import queue
from collections import Counter
import random


def set_phormoneTrail(pheromone_size, initial_value):
    
    trail = np.full((pheromone_size, pheromone_size), initial_value, dtype=float)
    
    return trail

def getRandomCity(my_cities):
    
    idx = np.random.choice(my_cities.shape[0])  # Get a random index
    city = my_cities[idx]  # Get the selected element
    my_cities = np.delete(my_cities, idx)  # Remove the element
    return city, my_cities  # Return the removed element and the updated array

def calculate_probs_next_city(my_cities, city, tsp_data, pheromone_trail):
    
    #alpha = 0.7
    #beta = 0.3
    alpha = 1.0
    beta = 2.0
    
    d = tsp_data[city] # get distances from city to all cities
    d = d[my_cities] # keep only distances of available cities in my_cities
    n = 1/d
    
    t = pheromone_trail[city] # get pheromone from city to all cities
    t = t[my_cities]  # keep only pheromones of available cities in my_cities
    
    my_sum = np.sum(np.power(t, alpha) * np.power(n, beta))  # Element-wise multiplication, then sum
    
    probs = [((t[i]**alpha)*(n[i]**beta))/my_sum for i in range(my_cities.size)]
    
    return probs

def update_my_cities(my_cities, city):
    # Find index of the value to remove
    idx = np.where(my_cities == city)[0]
    my_cities = np.delete(my_cities, idx)  # Remove by index
    
    return my_cities
    
def build_ant(tsp_data, pheromone_trail):
    ant = []
    
    number_of_cities = pheromone_trail.shape[0]
    my_cities = np.random.choice(range(number_of_cities), number_of_cities, replace=False) #cities in [0,length]
    
    city, my_cities = getRandomCity(my_cities)
    ant.append(city)
    
    while my_cities.size !=0:
        
        probs = calculate_probs_next_city(my_cities, city, tsp_data, pheromone_trail)
        #select a city according to probs
        city = np.random.choice(my_cities, 1, p=probs)[0]
        ant.append(city)
        # remove selected city from cities
        my_cities = update_my_cities(my_cities, city)  # Remove the element
        
    return ant    

def generate_ants(colony_size, pheromone_trail):
    
    colony = []
    for _ in range(colony_size):
        colony.append(build_ant(tsp_data, pheromone_trail))
        
    return colony

def getQuality(ant):
    cost = 0
    for i in range(len(ant)-1):
        cost = cost + tsp_data[ant[i]][ant[i+1]]
    
    cost = cost + tsp_data[ant[-1]][ant[0]]
    quality = 1/cost
    return quality

def evaluate_colony(ants):
    
    quality_ants = []
    
    for index in range(len(ants)):
        quality_ants.append(getQuality(ants[index]))
        
    return np.array(quality_ants)

#def update_pheromone_trail(ant, quality, pheromone_trail):
    
    #for i in range(len(ant)-1):
        #pheromone_trail[i][i+1] = pheromone_trail[i][i+1] + quality
    
    #return pheromone_trail
    
def update_pheromone_trail(ant, quality, pheromone_trail):
    # Mise à jour des phéromones sur chaque arête du chemin, y compris le retour
    for i in range(len(ant) - 1):
        pheromone_trail[ant[i]][ant[i+1]] += quality
        pheromone_trail[ant[i+1]][ant[i]] += quality  # Optionnel si le graphe est symétrique
    pheromone_trail[ant[-1]][ant[0]] += quality
    pheromone_trail[ant[0]][ant[-1]] += quality
    return pheromone_trail

def objectivefunction(ant):
    cost = 0
    for i in range(len(ant)-1):
        cost = cost + tsp_data[ant[i]][ant[i+1]]
    
    cost = cost + tsp_data[ant[-1]][ant[0]]
    return cost        

In [3]:
def AntColony_TSP():
    colony_size = 200 # must be a power of 2 for code optimization
    pheromone_size = 17
    intial_pheromone_value = 100
    evap_value = 0.5
    elitism_size = 100
    stop = 50
   
    pheromone_trail = set_phormoneTrail(pheromone_size, intial_pheromone_value)
    
    
    for iterations in range(stop):
        
        ants = generate_ants(colony_size, pheromone_trail)
        
        #Evaporation
        pheromone_trail = (1-evap_value)*pheromone_trail
    
        # Prepare Reinforcement
        quality_ants = evaluate_colony(ants) # big value for good ants
    
        # elitism
        indices = np.argpartition(-quality_ants, elitism_size)[:elitism_size] # - to get biggest values in place of smallest values
        best_ants = [ants[indices[index]] for index in range (elitism_size)] # keep n best ants
        best_qualities = [quality_ants[indices[index]] for index in range (elitism_size)]

        # leave pheromone
        for index in range(elitism_size):
            pheromone_trail = update_pheromone_trail(best_ants[index], best_qualities[index], pheromone_trail) 
        
        if(iterations == 0):
            best_ant = best_ants[0]
            best_cost = objectivefunction(best_ant)
        else:
            ant = best_ants[0]
            cost = objectivefunction(ant)
            if (cost<best_cost):
                best_ant = ant
                best_cost = cost
        print([int(city) for city in best_ants[0]], objectivefunction(best_ants[0]))
        #print(pheromone_trail)
        
    return best_ant, best_cost
                
 

In [4]:
result = AntColony_TSP()
print("\nMeilleure route trouvée:", [int(city) for city in result[0]])
print("Coût de la meilleure route:", result[1])

[2, 14, 16, 5, 6, 7, 12, 3, 0, 15, 11, 8, 4, 10, 1, 9, 13] 2397.0
[11, 8, 12, 6, 5, 7, 16, 10, 4, 1, 9, 14, 13, 2, 0, 3, 15] 2392.0
[1, 4, 10, 2, 13, 14, 7, 16, 6, 5, 3, 12, 0, 15, 11, 8, 9] 2321.0
[7, 5, 16, 6, 3, 12, 0, 13, 14, 2, 4, 10, 9, 1, 8, 11, 15] 2422.0
[11, 15, 0, 5, 7, 3, 12, 16, 6, 13, 14, 2, 10, 4, 1, 9, 8] 2340.0
[12, 6, 16, 7, 5, 2, 13, 14, 4, 10, 9, 1, 8, 11, 15, 3, 0] 2297.0
[8, 3, 12, 6, 7, 5, 16, 13, 14, 2, 10, 4, 1, 9, 0, 15, 11] 2243.0
[2, 14, 13, 5, 16, 7, 6, 12, 3, 0, 15, 11, 8, 10, 1, 9, 4] 2266.0
[5, 7, 16, 3, 6, 12, 0, 15, 11, 8, 13, 2, 10, 9, 1, 4, 14] 2348.0
[3, 12, 6, 7, 5, 16, 13, 14, 2, 10, 4, 1, 9, 15, 11, 8, 0] 2352.0
[2, 14, 5, 7, 16, 6, 12, 3, 0, 15, 11, 8, 4, 10, 1, 9, 13] 2318.0
[6, 16, 5, 7, 12, 3, 13, 14, 2, 9, 10, 4, 1, 8, 11, 15, 0] 2322.0
[8, 11, 15, 0, 3, 12, 6, 16, 7, 5, 2, 13, 14, 10, 4, 9, 1] 2246.0
[3, 12, 6, 7, 5, 16, 13, 14, 2, 10, 4, 9, 1, 8, 11, 15, 0] 2178.0
[9, 1, 4, 10, 2, 14, 13, 16, 5, 7, 6, 0, 12, 3, 15, 11, 8] 2152.0
[3, 12, 0,